# Test Generation Multidata

In this notebook, we generate the different examples for testing the method of OutagePipeline class

In [1]:
from anomalydetector import MultiDataHandler,OutagePipeline
import pandas as pd
import os
import numpy as np
from pathlib import Path

### Data loading

In [7]:
folder_path = Path.cwd().parent.parent / "tests" / "fixtures" / "anomaly_detection"
sites = ['01_TAEHC1041562.csv', '02_TAEHC1041811.csv', '03_ZT164285000441C0745.csv']
dfs = {}
for filename in sites:
    if filename.endswith(".csv"):
        file_path = folder_path / filename
        dfs[filename] = pd.read_csv(file_path)


### Data cleaning

We firstly use the functionnalities of MultiDataHandler to clean the data and select a small number of valid days.

In [4]:
dhs = MultiDataHandler(data_frames=list(dfs.values()),datetime_col='measured_on')
dhs.align()
dhs.dilate(ndil = 51)

Aligning datasets: 100%|██████████| 3/3 [00:00<00:00, 352.16it/s]


In [5]:
nd_day = 20
dil_mat_reduced = {}
common_day_reduced = dhs.common_days[:nd_day]
for site in dhs.dil_mat :
    dil_mat_reduced[site] = dhs.dil_mat[site][:,:nd_day]


### Reduced multidatahandler

In [6]:
target = 'ac_power_inv_30339'
dhs_reduced = MultiDataHandler(data_frames=[],datetime_col='measured_on')
dhs_reduced.dil_mat = dil_mat_reduced
dhs_reduced.common_days = common_day_reduced
dhs_reduced.target = target
dhs_reduced.generate_failure(target=dhs_reduced.target)

We save the data to reconstruct the MultiDataHandler aligned

In [ ]:
for filename in dil_mat_reduced :
    np.savetxt(folder_path / f"one_month_{filename}.csv",dil_mat_reduced[filename],delimiter=",")
np.savetxt(folder_path /"one_month_failure.csv",dhs_reduced.failure_mat,delimiter=",")
np.savetxt(folder_path / "common_day.csv",common_day_reduced,delimiter=",")

In [11]:
model = OutagePipeline(dhs_reduced.dil_mat.keys(),
                dhs_reduced.ndil(),
                target=target,
                quantiles=np.array([0.05, 0.5,0.95]),
                solver_quantiles='clarabel')

In [12]:
model.fit_quantiles(dhs_reduced)

100%|█████████████████████████████████████████████| 3/3 [00:06<00:00,  2.20s/it]


We save the data to reconstruct the OutagePipeline object after the quantile fit. 

In [ ]:
for filename in model.quantile_train :
    np.savetxt(folder_path / f"one_month_transform_{filename}.csv",model.quantile_train[filename],delimiter=",")
np.savetxt(folder_path / f"one_month_transform_failure.csv",model.quantile_failure,delimiter=",")

In [14]:
model.fit_linear()

We save the coefficients of the final linear regression

In [ ]:
np.savetxt(folder_path / "linear_coef.csv",model.linear_coeff,delimiter=",")